# Merging DataFrames

In [85]:
import pandas as pd

## Our Dataset
- Our datasets are spread across multiple files in this section. Each file has a `restaurant_` prefix.
- The `customers.csv` file stores our restaurant's customers.
- The `foods.csv` file stores our restaurant's menu items.
- The `week_1_sales` and `week_2_sales` files store our orders.

In [86]:
foods = pd.read_csv('restaurant_foods.csv')
customers = pd.read_csv('restaurant_customers.csv')
week1 = pd.read_csv('restaurant_week_1_sales.csv')
week2 = pd.read_csv('restaurant_week_2_sales.csv')

## The pd.concat Function I
- The `concat` function concatenates one **DataFrame** to the end of another.
- The original index labels will be kept by default. Set `ignore_index` to True to generate a new index.
- The `keys` parameter create a **MultiIndex** using the specified keys/labels.

In [87]:
week1.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [88]:
week2.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [89]:
# incluir week1 e week2 em 01 DF:
pd.concat([week1,week2], keys=["Week 1","Week 2"])

Customer ID  Food ID
Week 1 0            537        9
       1             97        4
       2            658        1
       3            202        2
       4            155        9
...                 ...      ...
Week 2 245          783       10
       246          556       10
       247          547        9
       248          252        9
       249          249        6

[500 rows x 2 columns]

In [90]:
pd.concat([week1,week2], ignore_index=True)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


## The pd.concat Function II
- Pandas will concatenate the **DataFrames** along the row/index axis.
- Pandas will include all columns that exist in either **DataFrame**. If there are no matching values, pandas will use `NaN` values.
- We can pass the `axis` parameter an argument of `"columns"` to concatenate on the column axis.

In [91]:
df1 = pd.DataFrame([1,2,3], columns=['A'])
df1

,A
0,1
1,2
2,3


In [92]:
df2 = pd.DataFrame([4,5,6], columns=['B'])
df2

,B
0,4
1,5
2,6


In [93]:
pd.concat([df1,df2], axis='index')

,A,B
0,1.0,NaN
1,2.0,NaN
2,3.0,NaN
0,NaN,4.0
1,NaN,5.0
2,NaN,6.0


In [94]:
pd.concat([df1,df2], axis='columns')

,A,B
0,1,4
1,2,5
2,3,6


## Left Joins
- The `merge` method joins two **DataFrames** together based on shared values in a column or an index.
- A left join merges one **DataFrame** into another based on values in the first one.
- The "left" **DataFrame** is the one we invoke the `merge` method on.
- If the left **DataFrame's** value is not found in the right **DataFrame**, the row will hold `NaN` values.<br>
<img src="SQL_Joins.png" width="800" height="800"/>

In [95]:
week1.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [96]:
foods.head()

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99
3,4,Quesadilla,4.25
4,5,Pizza,2.49


In [97]:
week1.merge(right=foods, how='left', on='Food ID')

,Customer ID,Food ID,Food Item,Price
0,537,9,Donut,0.99
1,97,4,Quesadilla,4.25
2,658,1,Sushi,3.99
3,202,2,Burrito,9.99
4,155,9,Donut,0.99
...,...,...,...,...
245,413,9,Donut,0.99
246,926,6,Pasta,13.99
247,134,3,Taco,2.99
248,396,6,Pasta,13.99


## The left_on and right_on Parameters
- The `left_on` and `right_on` parameters designate the column names from each **DataFrame** to use in the merge.

In [98]:
week2.head()

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10
3,189,5
4,267,3


In [99]:
customers.head()

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive
3,4,Steven,Evans,Male,Fatz,Registered Nurse
4,5,Judy,Morrison,Female,Demivee,Legal Assistant


In [100]:
# trazer a direita a tabela clientes, sendo na esquerta a Primary Key 'Customer ID' e na direita 'ID'
week2.merge(right=customers, how='left', left_on='Customer ID', right_on='ID').head()

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,688,10,688,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,813,7,813,Johnny,Walker,Male,Kayveo,Developer II
2,495,10,495,Deborah,Little,Female,Babbleblab,VP Accounting
3,189,5,189,Roger,Gordon,Male,Skilith,Operator
4,267,3,267,Matthew,Wood,Male,Agimba,Product Engineer


In [101]:
week2.merge(right=customers, how='left', left_on='Customer ID', right_on='ID').drop(['ID','Customer ID'], axis='columns').head()

,Food ID,First Name,Last Name,Gender,Company,Occupation
0,10,Carl,Williamson,Male,Thoughtmix,Graphic Designer
1,7,Johnny,Walker,Male,Kayveo,Developer II
2,10,Deborah,Little,Female,Babbleblab,VP Accounting
3,5,Roger,Gordon,Male,Skilith,Operator
4,3,Matthew,Wood,Male,Agimba,Product Engineer


## Inner Joins I
- Inner joins merge two tables based on *shared*/*common* values in columns.
- If only one **DataFrame** has a value, pandas will exclude it from the final results set.
- If the same ID occurs multiple times, pandas will store each possible combination of the values.
- The design of the join ensures that the results will be the same no matter what **DataFrame** the `merge` method is invoked upon.<br>
<img src="SQL_Joins.png" width="800" height="800"/>

In [102]:
week1[week1['Customer ID'] == 155]

,Customer ID,Food ID
4,155,9
17,155,1


In [103]:
week2[week2['Customer ID'] == 155]

,Customer ID,Food ID
208,155,3


In [104]:
# Verificar os clientes que vieram tanto na semana 1, como na semana 2: INNER JOIN
week1.merge(week2, how='inner', on='Customer ID', suffixes=[' - Week 1', ' - Week 2'])

,Customer ID,Food ID - Week 1,Food ID - Week 2
0,537,9,5
1,155,9,3
2,503,5,8
3,503,5,9
4,155,1,3
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7


## Inner Joins II
- We can pass multiple arguments to the `on` parameter of the `merge` method. Pandas will require matches in both columns across the **DataFrames**.

In [105]:
week1.merge(week2, how='inner', on=['Customer ID','Food ID'])

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,922,1
6,21,4
7,578,5
8,578,5


In [106]:
condition_01 = week1['Customer ID'] == 21
condition_02 = week1['Food ID'] == 4
week1[condition_01 & condition_02]

,Customer ID,Food ID
101,21,4
212,21,4


In [107]:
condition_01 = week2['Customer ID'] == 21
condition_02 = week2['Food ID'] == 4
week2[condition_01 & condition_02]

,Customer ID,Food ID
30,21,4


## Full/Outer Join
- A **full/outer** joins values that are found in either **DataFrame** or both **DataFrames**.
- Pandas does not mind if a value exists in one **DataFrame** but not the other.
- If a value does not exist in one **DataFrame**, it will have a `NaN`.

<img src="SQL_Joins.png" width="800" height="800"/>

In [108]:
merged = week1.merge(week2, how='outer', on='Customer ID', suffixes=[' - Week 1', ' - Week 2'], indicator=True)
merged

,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
...,...,...,...,...
449,985,5.0,NaN,left_only
450,991,2.0,NaN,left_only
451,994,NaN,2.0,right_only
452,996,NaN,10.0,right_only


In [109]:
merged['_merge'].value_counts()

_merge
right_only    197
left_only     195
both           62
Name: count, dtype: int64

In [110]:
merged[merged['_merge'].isin(['left_only','right_only'])]

,Customer ID,Food ID - Week 1,Food ID - Week 2,_merge
0,3,2.0,NaN,left_only
1,8,NaN,6.0,right_only
2,10,2.0,NaN,left_only
3,13,NaN,2.0,right_only
4,20,1.0,NaN,left_only
...,...,...,...,...
449,985,5.0,NaN,left_only
450,991,2.0,NaN,left_only
451,994,NaN,2.0,right_only
452,996,NaN,10.0,right_only


## Merging by Indexes with the left_index and right_index Parameters
- Use the `on` parameter if the column(s) to be matched on have the same names in both **DataFrames**.
- Use the `left_on` and `right_on` parameters if the column(s) to be matched on have different names in the two **DataFrames**.
- Use the `left_index` or `right_index` parameters (set to True) to if the values to be matched on are found in the index of a **DataFrame**.

In [111]:
foods = pd.read_csv('restaurant_foods.csv', index_col='Food ID')
customers = pd.read_csv('restaurant_customers.csv', index_col='ID')
week1 = pd.read_csv('restaurant_week_1_sales.csv')
week2 = pd.read_csv('restaurant_week_2_sales.csv')

In [112]:
week1.head()

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9


In [113]:
customers.head()

,First Name,Last Name,Gender,Company,Occupation
ID,,,,,
1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
3,Roger,Black,Male,Tagfeed,Account Executive
4,Steven,Evans,Male,Fatz,Registered Nurse
5,Judy,Morrison,Female,Demivee,Legal Assistant


In [117]:
week1.merge(
    right=customers,
    how='left', 
    left_on='Customer ID', 
    right_index=True
    ).merge(
        foods, 
        how='left', 
        left_on='Food ID', 
        right_index=True
        ).drop(['Food ID','Customer ID'], axis='columns')

,First Name,Last Name,Gender,Company,Occupation,Food Item,Price
0,Cheryl,Carroll,Female,Zoombeat,Registered Nurse,Donut,0.99
1,Amanda,Watkins,Female,Ozu,Account Coordinator,Quesadilla,4.25
2,Patrick,Webb,Male,Browsebug,Community Outreach Specialist,Sushi,3.99
3,Louis,Campbell,Male,Rhynoodle,Account Representative III,Burrito,9.99
4,Carolyn,Diaz,Female,Gigazoom,Database Administrator III,Donut,0.99
...,...,...,...,...,...,...,...
245,Diane,Bailey,Female,Wikibox,Technical Writer,Donut,0.99
246,Anne,Wagner,Female,Skyba,Legal Assistant,Pasta,13.99
247,Diana,Hall,Female,Quinu,Financial Advisor,Taco,2.99
248,Juan,Romero,Male,Zoonder,Analyst Programmer,Pasta,13.99


## The join Method
- The `join` method is a shortcut for concatenating two **DataFrames** when merging by index labels.

In [120]:
times = pd.read_csv('restaurant_week_1_times.csv')
times.head()

,Time of Day
0,14:54:59
1,20:55:17
2,01:16:22
3,16:17:26
4,19:26:11


In [122]:
week1.merge(times, how='left', left_index=True, right_index=True)

,Customer ID,Food ID,Time of Day
0,537,9,14:54:59
1,97,4,20:55:17
2,658,1,01:16:22
3,202,2,16:17:26
4,155,9,19:26:11
...,...,...,...
245,413,9,04:44:14
246,926,6,07:46:21
247,134,3,20:45:08
248,396,6,01:09:06


In [123]:
week1.join(times)

,Customer ID,Food ID,Time of Day
0,537,9,14:54:59
1,97,4,20:55:17
2,658,1,01:16:22
3,202,2,16:17:26
4,155,9,19:26:11
...,...,...,...
245,413,9,04:44:14
246,926,6,07:46:21
247,134,3,20:45:08
248,396,6,01:09:06
